## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

## 데이터 불러오기

특징값 데이터 불러와서 정상/고장 분리

In [ ]:
FeatureData = pd.read_csv('./ProcessdData/FeatureData', sep=',', header=None)
FeatureData.shape

P value 순위 데이터 불러오기

In [ ]:
P_value_Rank = pd.read_csv('./ProcessdData/P_value_Rank' , sep=',' , header=None)
P_value_Rank

## P value 기준 Feature 선정

In [ ]:
# StartRank 부터 Number 만큼의 Feature
StartRank = 1
Number = 30

SelectedFeatues = np.zeros((Number,FeatureData.shape[1]))

s = 0

for i in range(StartRank, StartRank+Number):
    
    index                 = int(P_value_Rank.iloc[i-1,0])
    SelectedFeatues[s,:]   = FeatureData.iloc[index,:].values
    s += 1

# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(SelectedFeatues)
FeatureSelected.shape

In [ ]:
FeatureSelected = np.transpose(FeatureSelected)
FeatureSelected.shape

## 데이터 정규화

In [ ]:
# 선정된 특징데이터에 대한 정규화
FeatureSelected_std = StandardScaler().fit_transform(FeatureSelected)
FeatureSelected_std.shape

## 선정된 Feature PCA

In [ ]:
# 10개 PC(Principal Component) 추출
pca = PCA(n_components = 10)
PC = pca.fit_transform(FeatureSelected_std)

# 설명된 분산의 비율
VarRatio = pca.explained_variance_ratio_
VarRatio = pd.DataFrame(np.round_(VarRatio,3))

# 설명된 분산의 비율 누적
CumVarRatio = np.cumsum(pca.explained_variance_ratio_)
CumVarRatio_df = pd.DataFrame(np.round_(CumVarRatio,3))

Result = pd.concat([VarRatio , CumVarRatio_df], axis=1)
Result


In [ ]:
# 누적값 그래프로 시각화
plt.plot(range(1,11), CumVarRatio*100, color='green', linestyle='-', marker='o')
plt.ylim(0,100)
plt.xlim(0,11)
plt.grid()
plt.xlabel('PC Number')
plt.ylabel('Percentage(%)')
plt.show()

## Selected Feature PCA (2차원) 기반 분포 시각화

In [ ]:
NoOfData = int(FeatureData.shape[1]/2)

plt.plot(PC[:NoOfData,0], PC[:NoOfData,1], color='b', linestyle='', marker='o', label='Normal')
plt.plot(PC[NoOfData:,0], PC[NoOfData:,1], color='r', linestyle='', marker='o', label='Abnormal')
plt.legend()
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()